# Convert GitHub Actions `.yaml` to Parallel Works ACTIVATE `.yaml`

ACTIVATE `.yaml`-based workflows have the same "feel" as GitHub Actions so there are many similarities. However, they are not the same. GitHub Actions has a much wider feature set and `.yaml`s that run on GitHub runners have the advantage of implicit context with many additional environment variables.

This notebook is a first draft, interactive tool for ingesting a GitHub Actions `.yaml`, clearing out some things that Parallel Works ACTIVATE definitely does not support, adding some boilerplate "header" information that is likely to be useful for ACTIVATE users, and keeping key workflow steps that ACTIVATE does support (namely the `jobs: steps: run:` framework and any explicit `env:` variables).

This notebook is a work in progress and not a complete solution hence it is a living document and meant for experimentation.

In [1]:
input_file="/home/sfgary/ubuntu-ci-x86_64-gnu.yaml"
output_file=input_file+".2pw.yaml"

In [2]:
# Dependencies may be already installed if running this notebook
# within a JupyterLab instance.
#!pip install pyyaml

In [3]:
import yaml

In [4]:
# Load the file you want to convert
with open(input_file) as stream:
    try:
        #print(yaml.safe_load(stream))
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [5]:
data.keys()

dict_keys(['name', True, 'concurrency', 'defaults', 'env', 'jobs'])

In [6]:
# Get rid of som GitHub Actions things that ACTIVATE does not use
data.pop('name')
data.pop(True)
data.pop('concurrency')
data.pop('defaults')

data.keys()

dict_keys(['env', 'jobs'])

In [8]:
data['jobs']['ubuntu-ci-c6a-x86_64-gnu-build']

{'runs-on': ['ubuntu-ci-c6a-x86_64'],
 'steps': [{'name': 'cleanup',
   'run': 'pwd\nls -lart\nset +e\nfind ./* -type d -exec chmod u+xw {} \\;\nset -e\nrm -fr *\n'},
  {'name': 'checkout',
   'uses': 'actions/checkout@v4',
   'with': {'submodules': True}},
  {'name': 'prepare-directories',
   'run': '# DH* REVERT ME AFTER MERGE\nmkdir -p ${BUILD_CACHE_PATH}\nmkdir -p ${SOURCE_CACHE_PATH}\n'},
  {'name': 'create-buildcache',
   'run': '# Get day of week to decide whether to use build caches or not\nDOW=$(date +%u)\n# Monday is 1 ... Sunday is 7\nif [[ $DOW == 7 ]]; then\n  export USE_BINARY_CACHE=false\n  echo "Ignore existing binary cache for creating buildcache environment"\nelse\n  export USE_BINARY_CACHE=true\n  echo "Use existing binary cache for creating buildcache environment"\nfi\n\n# Set up spack-stack\nsource ./setup.sh\n\ndeclare -a TEMPLATES=("unified-dev" "skylab-dev" "cylc-dev" "neptune-dev")\nfor TEMPLATE in "${TEMPLATES[@]}"; do\n  if [[ "${TEMPLATE}" == *"unified-dev"*

In [ ]:
# Change runs-on: to ssh:
# Get list of jobs, header_items first, before iterating
# because otherwise Python throws an error when size
# of the dictionary changes during the iteration.

jobs = list(data['jobs'].keys())
for job in jobs:

    header_items = list(data['jobs'][job].keys())
    for header_item in header_items:
        if header_item == "runs-on":
            data['jobs'][job]["ssh"] = data['jobs'][job]["runs-on"]
            data['jobs'][job]["ssh"] = list("remoteHost: ${{inputs.resource.ip}}")
            del data['jobs'][job]["runs-on"]


In [ ]:
data['jobs'][job].keys()

In [9]:
# Print out the final converted file
with open(output_file, 'w') as stream:
    try:
        yaml.dump(data, stream, default_style='literal')    # Write a YAML representation of data
    except yaml.YAMLError as exc:
        print(exc)
        
print(yaml.dump(data))    # Output the document to the screen.

env:
  BUILD_CACHE_PATH: /home/ubuntu/spack-stack/build-cache-new-spack-v1
  SOURCE_CACHE_PATH: /home/ubuntu/spack-stack/source-cache
jobs:
  ubuntu-ci-c6a-x86_64-gnu-build:
    runs-on:
    - ubuntu-ci-c6a-x86_64
    steps:
    - name: cleanup
      run: 'pwd

        ls -lart

        set +e

        find ./* -type d -exec chmod u+xw {} \;

        set -e

        rm -fr *

        '
    - name: checkout
      uses: actions/checkout@v4
      with:
        submodules: true
    - name: prepare-directories
      run: '# DH* REVERT ME AFTER MERGE

        mkdir -p ${BUILD_CACHE_PATH}

        mkdir -p ${SOURCE_CACHE_PATH}

        '
    - name: create-buildcache
      run: "# Get day of week to decide whether to use build caches or not\nDOW=$(date\
        \ +%u)\n# Monday is 1 ... Sunday is 7\nif [[ $DOW == 7 ]]; then\n  export\
        \ USE_BINARY_CACHE=false\n  echo \"Ignore existing binary cache for creating\
        \ buildcache environment\"\nelse\n  export USE_BINARY_CACHE=true\n